In [33]:
#https://stackoverflow.com/questions/52385655/unable-to-locate-package-google-drive-ocamlfuse-suddenly-stopped-working!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}!mkdir -p drive
!google-drive-ocamlfuse drive

--2019-10-26 17:46:02--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 91.189.89.223, 2001:67c:1560:8003::8004, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-10-26 17:46:03 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
··········
/bin/bash: google-drive-ocamlfuse: command not found
/bin/bash: google-drive-ocamlfuse: command not found


In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')#, force_remount=True)
curDir =('/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project')
os.chdir(curDir)
!cat /content/gdrive/My\ Drive/Colab\ Notebooks/DeepMovement_project
!pwd
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
cat: '/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project': Is a directory
/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project


Found GPU at: /device:GPU:0


In [39]:
tf.__version__
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13022        2145        7338           5        3539       10958
Swap:             0           0           0


# 1. Imports

In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time
import copy
import pickle
import random
from random import shuffle 

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
# from keras.utils.np_utils import to_categorical

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from torchtext.data import Example
import torchtext

torch.manual_seed(10)
# from google.colab import files
# src = list(files.upload().values())[0]
# open('UsedFunctions.py','wb').write(src)
from UsedFunctions import modelBuild
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

# 2. Methods

In [0]:
def seed_everything(seed=10):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def Regulate(framesInSeq):
    rem=[]
    for j in range(len(framesInSeq)):
        if framesInSeq[j] <= 7:
            if len(framesInSeq) <= 1:
                print("Error, very small file")
                break
            if j == 0 and (j+1) < len(framesInSeq):
                framesInSeq[j+1] += framesInSeq[j]
            elif j == len(framesInSeq)-1 and (j-1) >= 0:
                framesInSeq[j-1] += framesInSeq[j]
            else:
                if framesInSeq[j+1] < framesInSeq[j-1]:
                    framesInSeq[j+1] += framesInSeq[j]  
                else: framesInSeq[j-1] += framesInSeq[j]
            rem.append(j)
    framesInSeq=np.delete(framesInSeq,rem)
    return framesInSeq

def ReadData():
    """ ----- Very important to note the pathes ----- """
    # Getting training data
    df = pd.read_csv(os.path.join('.','Output','Activity_train.csv')) #read data
    df.dropna(axis=0, how='any', thresh=76, subset=None, inplace=True)
    # Getting consistant sequence of frames
    fr = df.copy()['Frame']
    target = df.copy()['Human']
    
    #dropping usless columns and columns with all their values are 0
    dropped = ['Human', 'V_title', 'Frame', 'L Foot_x_PCM', 'L Foot_y_PCM', 'R Foot_L',	'L Foot_L',	'Head & Neck_R',
               'Trunk_R',	'R Upper Arm_R',	'L Upper Arm_R',	'R Forearm_R',	'L Forearm_R',	'R Hand_R',	'L Hand_R',
               'R Thigh_R',	'L Thigh_R',	'R Shank_R',	'L Shank_R',	'R Foot_R',	'L Foot_R']
    df.drop(columns=dropped, inplace=True)
    
    seq_count = np.zeros(fr.shape, dtype=int)
    for i in range(1, len(fr.index)):
        # assuming if there is > 30 frames gap, the scene is changed
        if np.abs(fr.values[i-1] - fr.values[i]) > 30:
            seq_count[i] = seq_count[i-1] + 1
        else:
            seq_count[i] = seq_count[i-1]
    framesInSeq = np.bincount(seq_count)
    
    framesInSeq = Regulate(framesInSeq)
    
    maxSeq = framesInSeq.max() #// 13
        
    nan_locations = np.isnan(np.float_(df.values)) # get locations of nan values to set to 0 later
    df.fillna(0, inplace=True) # convert NaN values to zeros
    
    array = df.values #conver df to array
    X = np.array(array, dtype='float')
    #target = array[:, 0] #split target data
    l = list(set(target.copy()))
    labelencoder = LabelEncoder()
    target = labelencoder.fit_transform(target)
    
    # get unique labels 
    unique_labels = dict(zip(labelencoder.fit_transform(l), l))
    
    #X = np.delete(X, np.s_[:1], 1) #drop target data 
    #X = np.float64(X)
    Y = np.array(target, dtype='int')
    #Y=Y.astype('int') # issue
    
    # Feature scaling 
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X[nan_locations]=0 # set location of nans to 0
    
    Data_X = []
    Data_Y = []
    Num = 0
    ind = 0
    for cLen in framesInSeq:
        #if clen > maxSeq:
        #    clen /= 3
        for i in range((cLen // maxSeq) + 1 * ((cLen % maxSeq)!=0)): # in normal situations i is always =0
            x = []
            for j in range(maxSeq):
                if (maxSeq*i + j) >= cLen:
                    x = np.tile(x, (maxSeq//len(x)+1, 1))[:maxSeq].tolist()
                    break
                else:
                    x.append(list(X[Num + maxSeq*i + j]))
                    y = Y[Num + maxSeq*i + j]
            Data_X.append(x)
            Data_Y.append(y) 
        Num += cLen
        ind += 1
    #Data_Y = to_categorical(Data_Y) # convert to one hot
    onehot_encoder = OneHotEncoder(categories='auto', sparse=False)
    Data_Y = onehot_encoder.fit_transform(np.array(Data_Y, dtype='float32').reshape(-1,1))
    maxSeq=50 ##@!
    return np.array(Data_X, dtype='float32')[:,:maxSeq], Data_Y, unique_labels, maxSeq


# 3. Preprocessing

## 3.1. Training initialization variables

In [0]:
# Variables
# sequence_size = 3
batch_size = 128
n_lr = 1e-3
decay = 1e-7
n_epochs = 300
lstm_out = [128,64,32]
dense_layer = 128
SEED = 10
n_splits = 7

## 3.2. Data manipulation

In [42]:
#reading data 
Trainpath = os.path.join(curDir, 'TrainData')
Data_X, Data_Y, classes, sequence_size = ReadData()
# shuffling data
Z= list(zip(Data_X, Data_Y))
shuffle(Z)
Data_X, Data_Y = zip(*Z)
Data_X, Data_Y = np.array(Data_X), np.array(Data_Y)
X_train, X_test, y_train, y_test = train_test_split(Data_X, Data_Y, test_size=0.10, random_state=42)
Cats = len(classes)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(classes)
categories = [classes[i] for i in sorted(classes.keys())]
print(categories)

(1027, 50, 79) (1027, 3)
(115, 50, 79) (115, 3)
{2: 'walking', 1: 'running', 0: 'jogging'}
['jogging', 'running', 'walking']


In [0]:
class MyDataset(Dataset):
    def __init__(self,dataset):
        self.dataset = dataset
    def __getitem__(self,index):
        data,target = self.dataset[index]
        return data,target,index
    def __len__(self):
        return len(self.dataset)

# 4. Build LSTM model

In [0]:
class LSTMMove(nn.Module):

    def __init__(self, input_shape, lstm_out, dense_out, out_len):
        super(LSTMMove, self).__init__()
        self.hidden_dim = lstm_out[-1]

        #self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm1 = nn.LSTM(input_shape, lstm_out[0], 3, batch_first=True, dropout=0.4)
        # self.lstm2 = nn.LSTM(lstm_out[0], lstm_out[1], batch_first=True, dropout=0.4)
        # self.lstm3 = nn.LSTM(lstm_out[1], lstm_out[2], batch_first=True, dropout=0.4)
        # The linear layer that maps from hidden state space to tag space
        self.dense= nn.Linear(lstm_out[0], dense_out)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(0.4)
        self.out = nn.Linear(dense_out, out_len)
        self.softmax = nn.Softmax()

    def forward(self, inputs):
        x, (h, c) = self.lstm1(inputs)
        #x, _ = nn.utils.rnn.pad_packed_sequence(x)
        x = self.drop(x)
        # x, _ = self.lstm2(x)
        # x, _ = self.lstm3(x)
        x = self.relu(self.dense(x))
        x = self.drop(x)
        x = self.out(x)
        x = self.softmax(x)
        return x[:,-1].view(len(x), -1)
# def modelBuild2(input_shape, lstm_out, dense_out):
    

In [9]:
x_t = torch.tensor(X_test, dtype=torch.float32).cuda()
y_t = torch.tensor(y_test, dtype=torch.float32).cuda()
test = MyDataset(torch.utils.data.TensorDataset(x_t, y_t))
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)
len(test_loader)
for i, (x_batch, y_batch, index) in enumerate(test_loader):
    print(i, x_batch.shape, y_batch.shape, index.shape)
# y_b = torch.tensor(np.random.randn(115,3), dtype=torch.float32).cuda()
# (y_t.argmax(1)==y_b.argmax(1)).sum().cpu().numpy()/len(y_t)

0 torch.Size([115, 50, 79]) torch.Size([115, 3]) torch.Size([115])


# 5. Training 

## 5.1. Initiation

In [0]:
def pytorch_model_run_cv(x_train,y_train,features,x_test, y_test, model_obj, feats = False,clip = True):
    seed_everything()
    avg_losses_f = []
    avg_val_losses_f = []
    avg_accs_f = []
    avg_val_accs_f = []
    # matrix for the out-of-fold predictions
    train_preds = np.zeros((len(x_train),Cats))
    # matrix for the predictions on the test set
    test_preds = np.zeros((len(x_test), Cats))
    splits = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED).split(x_train, y_train[:,1])
    x_test = torch.tensor(x_test, dtype=torch.float32).cuda()
    y_test = torch.tensor(y_test, dtype=torch.float32).cuda()
    test = MyDataset(torch.utils.data.TensorDataset(x_test, y_test))
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    for i, (train_idx, valid_idx) in enumerate(splits):
        seed_everything(i*1000+i)
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        if feats:
            features = np.array(features)
        x_train_fold = torch.tensor(x_train[train_idx.astype(int)], dtype=torch.float32).cuda()
        y_train_fold = torch.tensor(y_train[train_idx.astype(int), np.newaxis], dtype=torch.float32).cuda()
        if feats:
            kfold_X_features = features[train_idx.astype(int)]
            kfold_X_valid_features = features[valid_idx.astype(int)]
        x_val_fold = torch.tensor(x_train[valid_idx.astype(int)], dtype=torch.float32).cuda()
        y_val_fold = torch.tensor(y_train[valid_idx.astype(int), np.newaxis], dtype=torch.float32).cuda()

        model = copy.deepcopy(model_obj)

        model.cuda()

        loss_fn = nn.BCELoss()#(reduction='sum')

        step_size = 300
        base_lr, max_lr = n_lr/5, n_lr   
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                                 lr=max_lr)

        ################################################################################################
        scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr,
                   step_size_up=step_size, mode='exp_range',
                   gamma=0.99994, cycle_momentum=False)
        ###############################################################################################
        #print('xtrain',X_train[0])
        #print('xtrainF',x_train_fold[0])

        train = MyDataset(torch.utils.data.TensorDataset(x_train_fold, y_train_fold))
        #print('train',train[0])
        valid = MyDataset(torch.utils.data.TensorDataset(x_val_fold, y_val_fold))

        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        #print('loader',train_loader[0])
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

        print(f'Fold {i + 1}')
        for epoch in range(n_epochs):
            start_time = time.time()
            model.train()

            avg_loss = 0.  
            avg_acc = 0.
            for i, (x_batch, y_batch, index) in enumerate(train_loader):
                y_batch = y_batch.view(len(y_batch), -1)
                if feats:       
                    f = kfold_X_features[index]
                    y_pred = model([x_batch,f])
                else:
                    y_pred = model(x_batch)

                if scheduler:
                    scheduler.step()

                # Compute and print loss.
                #print(y_pred.shape, y_batch.shape)
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                if clip:
                    nn.utils.clip_grad_norm_(model.parameters(),1)
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)
                acc =(y_pred.argmax(dim=1)==y_batch.argmax(dim=1)).sum().cpu().numpy()/len(y_pred)
                avg_acc += acc / len(train_loader)
            #avg_loss /= i+1
            #avg_acc /= i+1
                

            model.eval()

            valid_preds_fold = np.zeros((x_val_fold.size(0), Cats))
            test_preds_fold = np.zeros((len(x_test), Cats))

            avg_val_loss = 0.
            avg_val_acc = 0.
            for i, (x_batch, y_batch, index) in enumerate(valid_loader):
                y_batch = y_batch.view(len(y_batch), -1)
                if feats:
                    f = kfold_X_valid_features[index]            
                    y_pred = model([x_batch,f]).detach()
                else:
                    y_pred = model(x_batch).detach()

                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                val_acc =(y_pred.argmax(dim=1)==y_batch.argmax(dim=1)).sum().cpu().numpy()/len(y_pred)
                avg_val_acc += val_acc / len(valid_loader)
                valid_preds_fold[index] = y_pred.cpu().numpy()#[:, 0]
            #avg_val_loss /= i+1
            #avg_val_acc /= i+1

            elapsed_time = time.time() - start_time 
            print('Epoch {}/{} \t loss={:.4f} \t acc={:.4f} \t val_loss={:.4f} \t avg_val_acc={:.4f} \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_acc, avg_val_loss, avg_val_acc, elapsed_time))
        avg_losses_f.append(avg_loss)
        avg_val_losses_f.append(avg_val_loss) 
        avg_accs_f.append(avg_acc)
        avg_val_accs_f.append(avg_val_acc) 
        
        # predict all samples in the test set batch per batch
        for i, (x_batch, y_batch, index) in enumerate(test_loader):
            if feats:
                f = test_features[i * batch_size:(i+1) * batch_size]
                y_pred = model([x_batch,f]).detach()
            else:
                y_pred = model(x_batch).detach()

            test_preds_fold[i * batch_size:(i+1) * batch_size] = y_pred.cpu().numpy()#[:, 0]

        train_preds[valid_idx] = valid_preds_fold
        test_preds += test_preds_fold / n_splits

    print('All \t loss={:.4f} \t acc={:.4f} \t val_loss={:.4f} \t avg_val_acc={:.4f} \t '
          .format(np.average(avg_losses_f),np.average(avg_accs_f),np.average(avg_val_losses_f),np.average(avg_val_accs_f)))
    return model, train_preds, test_preds

In [28]:
beg = time.time()
model_obj = LSTMMove(X_train.shape[-1], lstm_out, dense_layer, Cats)
model_obj.load_state_dict(torch.load(os.path.join(curDir,'ModelData','Pytorch_Model','model_state_dict.pth')))
model, train_preds, test_preds = pytorch_model_run_cv(X_train,y_train,0,X_test,y_test, model_obj, feats = False,clip = True)
print("Training time = %.2fs"%(time.time()-beg))

Fold 1
Epoch 1/300 	 loss=1.3503 	 acc=0.9118 	 val_loss=1.1236 	 avg_val_acc=0.8898 	 time=0.35s
Epoch 2/300 	 loss=1.3517 	 acc=0.9015 	 val_loss=1.0360 	 avg_val_acc=0.9122 	 time=0.34s
Epoch 3/300 	 loss=1.3373 	 acc=0.9149 	 val_loss=1.0446 	 avg_val_acc=0.9015 	 time=0.34s
Epoch 4/300 	 loss=1.3247 	 acc=0.9262 	 val_loss=1.0233 	 avg_val_acc=0.9463 	 time=0.34s
Epoch 5/300 	 loss=1.3210 	 acc=0.9332 	 val_loss=1.0434 	 avg_val_acc=0.9122 	 time=0.34s
Epoch 6/300 	 loss=1.3310 	 acc=0.9099 	 val_loss=1.0342 	 avg_val_acc=0.9239 	 time=0.34s
Epoch 7/300 	 loss=1.3166 	 acc=0.9297 	 val_loss=1.0402 	 avg_val_acc=0.9200 	 time=0.34s
Epoch 8/300 	 loss=1.3149 	 acc=0.9413 	 val_loss=1.0276 	 avg_val_acc=0.9463 	 time=0.34s
Epoch 9/300 	 loss=1.3272 	 acc=0.9252 	 val_loss=1.0293 	 avg_val_acc=0.9239 	 time=0.34s
Epoch 10/300 	 loss=1.3166 	 acc=0.9373 	 val_loss=1.0393 	 avg_val_acc=0.9424 	 time=0.34s
Epoch 11/300 	 loss=1.3304 	 acc=0.9208 	 val_loss=1.0719 	 avg_val_acc=0.8859 	 t

## 5.2. Train statistics

In [29]:
print('Average Train accuracy ',(train_preds.argmax(1)==y_train.argmax(1)).sum()/len(y_train))
print('Average Test accuracy ',(test_preds.argmax(1)==y_test.argmax(1)).sum()/len(y_test))

Average Train accuracy  0.8519961051606622
Average Test accuracy  0.8956521739130435


In [30]:
x_tt = torch.tensor(X_train, dtype=torch.float32).cuda()
y_tt = torch.tensor(y_train, dtype=torch.float32).cuda()
y_pp = model(x_tt).detach()
print('Predicted Train accuracy ',(y_pp.argmax(dim=1)==y_tt.argmax(dim=1)).sum().cpu().numpy()/len(y_pp))
x_t = torch.tensor(X_test, dtype=torch.float32).cuda()
y_t = torch.tensor(y_test, dtype=torch.float32).cuda()
y_p = model(x_t).detach()
print('Predicted Test accuracy ',(y_p.argmax(dim=1)==y_t.argmax(dim=1)).sum().cpu().numpy()/len(y_p))

Predicted Train accuracy  0.9532619279454723
Predicted Test accuracy  0.8521739130434782


In [31]:
print('Train statistics:')
print('accuracy %s' % accuracy_score(y_pp.argmax(dim=1).cpu(), y_tt.argmax(dim=1).cpu()))
print(classification_report(y_tt.argmax(dim=1).cpu(), y_pp.argmax(dim=1).cpu(),target_names=categories))
print('Test statistics:')
print('accuracy %s' % accuracy_score(y_p.argmax(dim=1).cpu(), y_t.argmax(dim=1).cpu()))
print(classification_report(y_t.argmax(dim=1).cpu(), y_p.argmax(dim=1).cpu(),target_names=categories))

Train statistics:
accuracy 0.9532619279454723
              precision    recall  f1-score   support

     jogging       0.96      0.93      0.94       347
     running       0.97      0.95      0.96       315
     walking       0.93      0.98      0.96       365

    accuracy                           0.95      1027
   macro avg       0.95      0.95      0.95      1027
weighted avg       0.95      0.95      0.95      1027

Test statistics:
accuracy 0.8521739130434782
              precision    recall  f1-score   support

     jogging       0.91      0.71      0.80        42
     running       0.81      0.93      0.87        28
     walking       0.84      0.93      0.88        45

    accuracy                           0.85       115
   macro avg       0.85      0.86      0.85       115
weighted avg       0.86      0.85      0.85       115



# 6. Save necessary files

In [32]:
modelspath=os.path.join(curDir,'ModelData','Pytorch_Model')
if not os.path.exists(modelspath):
    os.makedirs(modelspath)
torch.save(model.state_dict(), os.path.join(modelspath,'model_state_dict.pth'))
torch.save(model, os.path.join(modelspath,'model.pth'))
print('Saved')

Saved


# Loading model

In [0]:
eval_model = copy.deepcopy(model_obj)
eval_model.load_state_dict(torch.load(os.path.join(curDir,'ModelData','Pytorch_Model','model_state_dict.pth')))
x_t = torch.tensor(X_test, dtype=torch.float32).cuda()
# y_t = torch.tensor(y_test, dtype=torch.float32).cuda()
y_p = eval_model(x_t).detach()
print('Predicted Test accuracy ',(y_p.argmax(dim=1)==y_t.argmax(dim=1)).sum().cpu().numpy()/len(y_p))

## Tests

In [0]:
model = LSTMMove(X_train.shape[1:], lstm_out, dense_layer, Cats)
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(inputs)
#     print(tag_scores)

for epoch in range(epochs):  # again, normally you would NOT do 300 epochs, it is toy data
    train = MyDataset(torch.utils.data.TensorDataset(X_train, y_train))
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    for x_batch, y_batch, index in train_loader:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
#         sentence_in = prepare_sequence(sentence, word_to_ix)
#         targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        preds = model(x_batch)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(preds, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    #test = MyDataset(torch.utils.data.TensorDataset(X_test, y_test))
    #test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)
    scores = model(X_test)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(scores)

In [0]:
Xs = np.random.randn(9,2)
ys = to_categorical(np.random.randint(0,8,(9,1)))
skf = StratifiedKFold(n_splits=3)
print(skf.get_n_splits(Xs, ys))
print(skf)
for i, (train_idx, valid_idx) in enumerate(skf.split(Xs[:,0], ys[:,0])):
    print("TRAIN:", train_idx, "TEST:", valid_idx)
    X_traing, X_testg = Xs[train_idx], Xs[valid_idx]
    y_traing, y_testg = ys[train_idx], ys[valid_idx]
X_testg

In [0]:
splits = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED).split(X_train, y_train[:,1])
for i, (x,y) in enumerate(splits):
    Xx,Yy = x,y
    #print(i)
print(X_train.shape)
print(X_train.dtype)

In [0]:
te = np.array(list(range(2,9))+[5])
te = te / te.sum()
(-te*np.log2(te)).sum()

2.893542397143181